**Project Summary:**

Telco customers are leaving for newer companies. Especially phone customers. Find a driver of the churn. A copy of the report will go to executives. Include enough documentation in your final notebook that someone could work through it without you.

**Goals:**
- Find drivers for customer churn at Telco
- Construct a ML classification model that accurately predicts customer churn. 

**Process/Pipeline:**  https://trello.com/b/vOXbVcbl





In [1]:
import pandas as pd
import acquire
import env
from acquire import get_telco_data

In [2]:
#Acquire the data from the telco_churn database on the Codeup data science database server

get_telco_data()


,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,internet_service_type_id,internet_service_type,contract_type_id,contract_type,payment_type_id,payment_type
0,0003-MKNFE,Male,0,No,No,9,Yes,Yes,1,No,...,2,59.90,542.4,No,1,DSL,1,Month-to-month,2,Mailed check
1,0013-MHZWF,Female,0,No,Yes,9,Yes,No,1,No,...,4,69.40,571.45,No,1,DSL,1,Month-to-month,4,Credit card (automatic)
2,0015-UOCOJ,Female,1,No,No,7,Yes,No,1,Yes,...,1,48.20,340.35,No,1,DSL,1,Month-to-month,1,Electronic check
3,0023-HGHWL,Male,1,No,No,1,No,No phone service,1,No,...,1,25.10,25.1,Yes,1,DSL,1,Month-to-month,1,Electronic check
4,0032-PGELS,Female,0,Yes,Yes,1,No,No phone service,1,Yes,...,3,30.50,30.5,Yes,1,DSL,1,Month-to-month,3,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9950-MTGYX,Male,0,Yes,Yes,28,Yes,No,3,No internet service,...,4,20.30,487.95,No,3,None,3,Two year,4,Credit card (automatic)
7039,9953-ZMKSM,Male,0,No,No,63,Yes,Yes,3,No internet service,...,2,25.25,1559.3,No,3,None,3,Two year,2,Mailed check
7040,9964-WBQDJ,Female,0,Yes,No,71,Yes,Yes,3,No internet service,...,4,24.40,1725.4,No,3,None,3,Two year,4,Credit card (automatic)
7041,9972-EWRJS,Female,0,Yes,Yes,67,Yes,No,3,No internet service,...,3,19.25,1372.9,No,3,None,3,Two year,3,Bank transfer (automatic)


In [ ]:
I want to:
- drop:
    - customer_id
- encode:
    - partner
    - dependents
    - phone_service
- make dummies:
    - gender
    - multiple_lines
    - online_security
    - online_backup
    - device_protection
    - tech_support
    - streaming_tv
    
- rename col:
    - tenure:tenure_months
    - internet_service_type_id:int_serv_type

In [ ]:
Notes for prep:  
df.total_charges = pd.to_numeric(df.total_charges, errors='coerce').astype('float64')

df.total_charges = df.total_charges.fillna(value=0)